# MLP를 위한 Class 설계

객체지향 프로그래밍은 변수와 알고리즘을 객체별로 모아 정리해서 프로그램의 가독성과 확장성을 높이는 발전된 프로그래밍 기법이다.

## Data 분할 : 학습, 검증 평가


딥러닝은 대표적인 데이터 중심 인공지능 방법론이다. 데이터 중심 인공지능 방법론에서 인공지능 프로그램은 필요한 지식이나 패턴을 스스로 학습해 문제를 해결한다.

반면 1980 인공지능 붐때에는 지식 중심 인고지능 방법론 연구가 활발했다. 전문 분야 지식(knowledge)를 찾아내 이를 프로그램에 직접 반영하는 방법이다. 원시 데이터로 주어지는 예제는 프로그램이 학습할 대상이 아니라 인공지능 개발자가 지식을 찾아내야 할 대상이었다. 하지만 지식을 찾아내어 프로그램으로 표현하는 일이 생각보다 너무 까다로운 문제였꼬 상충하는 지식을 어떻게 조율할지 마땅치 않아 장난감 수준을 벗어나는 프로그램을 만들기 어려웠다.

지식 중심의 접근 방법과 달리 데이터 중심 접근에서는 프로그램이 문제 해결에 필요한 정보를 스스로 학습하여 찾아낸다. 이러한 딥러닝 모델을 개발하려면 많은 데이터가 필요하다. 전문 지식을 갖춘 사람은 비교적 적은 데이터로도 많은 지식 정보를 추출해내지만 프로그램은 그렇지 못하기에 지식 중심의 접근에 비해 훨씬 많은 데이터가 필요하다. 또한 딥러닝 학습을 진행하려면 입력과 함께 출력이 어떠해야 할지를 알려주는 모범 답안도 제공되어야 한다. 이 모범 답안 정보를 label 정보라고 이야기 하는데, 신경망 학습은 데이터 입력을 가지고 얻어낸 출력을 이 label 정보와 비교하는 데에서 시작한다.

데이터에 label 정보를 추가하는 과정은 지겨운 반복 노동이라 구축 비용이 크다. 이런 부담을 덜고자 비지도학습이라는 접근 방법도 등장했다. 비지도학습은 레이블 정보 없이 입력을 출력, 즉 모범 답안을 삼아 학습을 수행한다. 그러나 이러한 비지도 학습은 label 정보를 이용하는 지도 학습을 보조하는 역할을 할 뿐 비지도학습만으로 인공 신경망이 필요한 일을 하게 만들기는 기술상 어렵다.

일반적으로 데이터는 학습 재료가 되는 training data, 학습 과정에서 중간 점검에 사용되는 verification data, 학습 후에 학습 품질을 확인하는데 사용되는 test data로 나눈다. 그런데 deep learning에서 정말 중요한 것은 학습 과정에서의 성적보다 실전에서의 성적이다. 따라서 학습 데이터도 중요하지만 검증 데이터나 평가 데이터를 잘 활용하여 학습 결과를 검증하고 평가하여 실전 적용에 알맞은 모습으로 학습되고 있는지를 확인하는 과정도 이에 못지 않게 중요하다. 이 때문에 세가지 데이터는 서로 겹치지 않으면서 골고루 섞이게 준비하는 것이 중요하다.



### 시각화

시각화는 딥러닝 모델의 처리 능력이나 처리 결과를 문제의 성격을 살려 시각적으로 보여주는 기능이다. 이 기능은 딥러닝 모델이 어떻게 동작하는지 이해를 돕고 혹시 있을지 모를 잘못된 처리나 문제를 조기에 파악하는 데 도움을 준다.

시각화 기능은 문제 특성에 맞게 정의되어야 한다. 그래서 구체적인 시각화 방법은 모델 클래스보다는 각 데이터셋을 이루는 dataset 파생 클래스에 정의하자.

### 이미지 분류 문제와 꽃 이미지 분류 데이터셋

이미지 분류는 딥러닝 알고리즘이 가장 큰 성과를 내는 분야이다. CNN(Convolution Neural Network)가 두각을 나타내고 있지만 MLP로도 이미지 분류가 불가능한 것은 아니다. 하지만 모델 크기, 즉 파라미터의 수가 지나치게 많은 데도 결과의 품질이 합성곱 신경망보다 뒤떨어지는 경우가 많아 거의 이용되지 않는다. 우선 MLP로 이미지 분류 문제를 풀어볼 것이다.

사진 분류 학습에서 데이터마다 입력 크기가 다르면 일관성 있게 처리하기가 어렵다. 그래서 데이터셋 준비 단계에서 이미지 크기를 일치하도록 고쳐놓는 것이 보통이다.

컬러 이미지는 RGB 색상으로 표현할 수 있다. 따라서 컬러 이미지를 표현하기 위해서 채널 세 개로 RGB 색상을 각각 표현하는 방법이 널리 사용된다. 이에 비해 단색 이미지는 채널 하나로 명암만을 표현하면 도니다. 이미지는 보통 [행, 열, 채널]의 3차원 구조의 픽셀 정보로 표현된다. 차후 공부할 합성곱 신경망은 이미지 처리에 특화되어 이미지의 3차원 구조를 살려둔 채 처리하지만 MLP에서는 입력 이미지를 1차원 벡터로 변환하여 처리한다.

이미지를 다룰 때는 이미지 해상도 설정과 은닉 계층 구성에 따라 필요한 파라미터 수가 매우 커질 수 있다. 예를 들어 꽃 사진을 분류하는 프로그램에서 이미지(섬네일) 크기를 160 X 120 pixel로 정했다고 하자. 이때 이미지는 [160, 120, 3] 형태이다. 따라서 꽃 사진 분류를 위한 프로그램에서 입력 벡터는 57600이며 출력 벡터 크기는 분류 대상 꽃의 가짓수에 의해 결정된다. 정보 압축 측면에서 볼때 은닉 벡터는 입력 벡터 크기와 출력 벡터 크기 사이에서 몇 단계에 걸쳐 차차 줄여주는 것이 좋다. 이에 따라서 은닉 계층 폭을 [5000, 500, 50] 세 단계로 정했다면 이때 모델 구성에 필요한 파라미터 수는 출력 계층까지 고려할 때 $57600 X 5000 + 5000 X 500 + 500 X 50 + 50 X 5 + 5$ 이다. 거의 3억개에 달한다.

이처럼 많은 파라미터를 제대로 학습시키려면 데이터도 많이 필요하다. 준비한 사진데이터 4000개로는 파라미터 3억개를 학습시키기에는 턱없이 부족하다. 많은 수의 파라미터를 갖는 큰 모델에 데이터까지 많아지면 계산 부담이 2중으로 늘어난다. 이런 모델은 엄청난 메모리와 계산 시간을 소모하면서도 좋은 결과를 내지 못한다.